In [13]:
import mysql.connector
import psycopg2

In [14]:
def source():
    mydb = mysql.connector.connect(
      host='localhost',
      user='root',
      password='root',
      database='ecommerce'
    )
    
    return mydb

In [15]:
def dest():
    connection = psycopg2.connect(user="warehouse",
                                  password="warehouse",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="warehouse")
    return connection
    

In [18]:
def get_last_product_id(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("select product_id from products order by product_id DESC limit 1")
        return cursor.fetchone()[0]
    except (Exception, psycopg2.DataError) as error:
        print ("Error while creating PostgreSQL table", error)
    finally:
        #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [21]:
def get_last_category_id(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("select category_id from categories order by category_id DESC limit 1")
        return cursor.fetchone()[0]
    except (Exception, psycopg2.DataError) as error:
        print ("Error while creating PostgreSQL table", error)
    finally:
        #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [23]:
last_product_id = get_last_product_id(dest())
last_category_id = get_last_category_id(dest())

PostgreSQL connection is closed
PostgreSQL connection is closed


In [24]:
last_product_id, last_category_id

(3, 2)

In [49]:
def get_missing_products(db, last_product_id):
    cursor = db.cursor()
    print("last prod id {}".format(last_product_id))

    sql = '''
        SELECT
            p.id as product_id,
            p.code as name
        FROM
            ecommerce_product p 
        WHERE
            p.id > %s
    ;
    '''

    cursor.execute(sql, (last_product_id,))
    
    results = cursor.fetchall()
    return results
    

In [79]:
products = get_missing_products(source(), get_last_product_id(dest()))

PostgreSQL connection is closed
last prod id 21


In [80]:
from psycopg2.extras import execute_values
def insert_many_dest(connection, sql, items):
    try:
        cursor = connection.cursor()
        execute_values(cursor, sql, items)
        connection.commit()
    except (Exception, psycopg2.DataError) as error:
        print ("Error while creating PostgreSQL table", error)
    finally:
        #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [81]:
products = get_missing_products(source(), get_last_product_id(dest()))
insert_many_dest(dest(), 'Insert into products (product_id, name) VALUES %s', products)

PostgreSQL connection is closed
last prod id 21
PostgreSQL connection is closed


In [82]:
def extract():
    products = get_missing_products(source(), get_last_product_id(dest()))
    
    return (products)

In [83]:
def transform(data):
    return data

In [84]:
def load(data):
    (products) = data
    insert_many_dest(dest(), 'Insert into products (product_id, name) VALUES %s', products)

In [85]:
load(transform(extract()))


PostgreSQL connection is closed
last prod id 21
PostgreSQL connection is closed
